In [57]:
import csv
import requests
from bs4 import BeautifulSoup

## データの取得

In [13]:
PAGE_URL = "https://www.adana.co.jp/jp/contents/retailer/shop.html"
r = requests.get(PAGE_URL)
r.encoding = r.apparent_encoding # エンコーディングをUTF-8に設定

soup = BeautifulSoup(r.text)
soup.title

<title>販売特約店をさがす | ADA - SHOP</title>

In [88]:
# １番目の店舗情報を取得
soup.find("a", class_=["basic-tr", "basic-tr-link"])

<a class="basic-tr-link" href="https://www.iland-yamagami.com/" rel="noopener" target="_blank">
<div class="basic-td">
<p><span class="shopname">札幌ホームアクアリウム （アイランド やまがみ）<i class="fas fa-external-link-alt"></i></span><br/>
                   北海道札幌市清田区北野五条2丁目6-15 アイランドやまがみ内<br/>
<i class="fas fa-phone"></i>011-885-1355<br/>
					 営業時間：10:00-19:00<br/>定休日：年中無休
                      </p>
</div>
<div class="basic-td shop-td5">
<p>
<img alt="ADA NATURE AQUARIUM 150 アイテム以上在庫" height="42" src="img/icon/ada_naitem_icon.jpg" width="42"/>
<img alt="DOOA 製品を30アイテム以上を常時在庫" height="42" src="img/icon/ada_dooaitem_icon.jpg" width="42"/>
<img alt="佗び草、BIOみずくさの森などADAの水草・熱帯植物を常に在庫" height="42" src="img/icon/ada_plants_icon.jpg" width="42"/>
<img alt="ADAのレイアウト素材を常時在庫" height="42" src="img/icon/ada_layout_icon.jpg" width="42"/>
</p>
</div>
</a>

## データの整形

In [91]:
shop_list = []

for link in soup.find_all("a", class_=["basic-tr", "basic-tr-link"]):
    shop_info = {}
    shop_detail = link.div.p.get_text(',').split(',')
    shop_detail = [x for x in shop_detail if x != "\n"] # 空行を除去

    shop_info["name"] = shop_detail[0].strip()
    shop_info["url"] = link.get("href")
    shop_info["address"] = shop_detail[1].strip()
    shop_info["phone"] = shop_detail[2].strip()
    # 改行が1つ少ないデータがあるため、処理を分岐
    if len(shop_detail) >= 5:
        shop_info["open"] = shop_detail[3].strip() + " " + shop_detail[4].strip()
    elif len(shop_detail) == 4:
        shop_info["open"] = shop_detail[3].strip()
    else:
        pass

    shop_list.append(shop_info)

shop_list[0]

{'name': '札幌ホームアクアリウム （アイランド やまがみ）',
 'url': 'https://www.iland-yamagami.com/',
 'address': '北海道札幌市清田区北野五条2丁目6-15 アイランドやまがみ内',
 'phone': '011-885-1355',
 'open': '営業時間：10:00-19:00 定休日：年中無休'}

## CSV出力

In [92]:
with open("shop_list.csv", "w") as f:
    writer = csv.DictWriter(f, fieldnames = shop_list[0].keys())
    writer.writeheader()
    writer.writerows(shop_list)
